# Pricing Optimization 
## Table of contents
* [Overview](#section-1)
* [Dataset](#section-2)
* [Objective](#section-3)
* [Costs](#section-4)
* [Create a BigQuery dataset](#section-5)
* [Load the dataset from Cloud Storage](#section-6)
* [Data Analysis](#section-7)
* [Preprocess the data for training](#section-8)
* [Train the model using BigQuery ML](#section-9)
* [Generate forecasts from the model](#section-10)
* [Interpret the results to choose the best price](#section-11)
* [Clean Up](#section-12)


## Overview
<a name="section-1"></a>
This notebook demonstrates analysis of pricing optimization on [CDM Pricing Data](https://github.com/trifacta/trifacta-google-cloud/tree/main/design-pattern-pricing-optimization) and automating the workflow using Vertex AI Workbench's managed notebooks.

<b>Note</b>: This notebook is designed to run on managed notebooks instance of Vertex AI Workbench. Some components of this notebook may not work in other notebook environments.

## Dataset
<a name="section-2"></a>
The dataset used in this notebook is a part of the [CDM Pricing Data](https://github.com/trifacta/trifacta-google-cloud/blob/main/design-pattern-pricing-optimization/CDM_Pricing_large_table.csv) which consists of products sales information on specified dates.

## Objective
<a name="section-3"></a>
The objective of this notebook is to build a pricing optimization model using Vertex AI on GCP. The following steps have been followed in this usecase :  

- Load the required dataset from a Cloud Storage bucket.
- Analyze the fields present in the dataset.
- Process the data to build a model.
- Build a BigQuery ML forecast model on the processed data.
- Get forecasted values from the BigQuery ML model.
- Interpret the forecasts to identify best prices.
- Clean up.


## Costs
<a name="section-4"></a>
This tutorial uses the following billable components of Google Cloud:

- Vertex AI
- Bigquery
- Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Bigquery pricing](https://cloud.google.com/bigquery/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

### Import the required libraries and define constants


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.cloud import bigquery
from google.cloud.bigquery import Client

In [ ]:
DATASET = "[your-bigquery-dataset-id]"  # set the Bigquery dataset-id
TRAINING_DATA_TABLE = "[your-bigquery-table-id-to-store-the-training-data]"  # set the Bigquery table-id to store the training data

## Create a BigQuery dataset
<a name="section-5"></a>


#@bigquery
-- create a dataset in BigQuery

CREATE SCHEMA pricing_optimization
OPTIONS(
  location="us"
  )

## Load the dataset from Cloud Storage
<a name="section-6"></a>


In [ ]:
DATA_LOCATION = "gs://vertex_ai_managed_services_demo/pricing_optimization/CDM_Pricing_large_table.csv"
df = pd.read_csv(DATA_LOCATION)
print(df.shape)
df.head()

We will build a forecast model on this data and thus determine the best price for a product. For this type of model, we may not be using many fields but just the sales and price related ones. For the current execrcise, we will just focus on the following fields :
- `Product_ID`
- `Customer_Hierarchy`
- `Fiscal_Date`
- `List_Price_Converged`
- `Invoiced_quantity_in_Pieces`
- `Net_Sales`



## Data Analysis
<a name="section-7"></a>

First, we will explore the data and distributions.

Select the required columns from the dataframe.

In [ ]:
id_col = "Product_ID"
date_col = "Fiscal_Date"
categ_cols = ["Customer_Hierarchy"]
num_cols = ["List_Price_Converged", "Invoiced_quantity_in_Pieces", "Net_Sales"]

df = df[[id_col, date_col] + categ_cols + num_cols].copy()
df.head()

Check the column types and null values in the dataframe.

In [ ]:
df.info()

This data description reveals that there are no null values in the data. Also, the field `Fiscal_Date` which is a date field is loaded as an object type. 

Change the type of the date field to datetime.

In [ ]:
df["Fiscal_Date"] = pd.to_datetime(df["Fiscal_Date"], infer_datetime_format=True)

Plot the distributions for the categorical fields.

In [ ]:
for i in categ_cols:
    df[i].value_counts(normalize=True).plot(kind="bar")
    plt.title(i)
    plt.show()

Plot the distributions for the numerical fields.

In [ ]:
for i in num_cols:
    _, ax = plt.subplots(1, 2, figsize=(10, 4))
    df[i].plot(kind="box", ax=ax[0])
    df[i].plot(kind="hist", ax=ax[1])
    ax[0].set_title(i + "-Boxplot")
    ax[1].set_title(i + "-Histogram")
    plt.show()

Check maximum date and minimum date in Fiscal_Date column.

In [ ]:
print(df["Fiscal_Date"].max())
print(df["Fiscal_Date"].min())

Check the product distribution across each category.

In [ ]:
grp_cols = ["Customer_Hierarchy", "Product_ID"]
grp_df = df[grp_cols].groupby(by=grp_cols).count().reset_index()
grp_df.groupby("Customer_Hierarchy").nunique()

Check the percentage changes in the orders based on the percentage changes in the price.

In [ ]:
# aggregate the data
df_aggr = (
    df.groupby(["Product_ID", "List_Price_Converged"])
    .agg({"Fiscal_Date": min, "Invoiced_quantity_in_Pieces": sum, "Net_Sales": sum})
    .reset_index()
)
# rename the aggregated columns
df_aggr.rename(
    columns={
        "Fiscal_Date": "First_price_date",
        "Invoiced_quantity_in_Pieces": "Total_ordered_pieces",
        "Net_Sales": "Total_net_sales",
    },
    inplace=True,
)

# sort values chronologically
df_aggr.sort_values(by=["Product_ID", "First_price_date"], inplace=True)
df_aggr.reset_index(drop=True, inplace=True)

# add columns for previous values
df_aggr["Previous_List"] = df_aggr.groupby(["Product_ID"])[
    "List_Price_Converged"
].shift()
df_aggr["Previous_Total_ordered_pieces"] = df_aggr.groupby(["Product_ID"])[
    "Total_ordered_pieces"
].shift()

# average price change across sku's
df_aggr["price_change_perc"] = (
    (df_aggr["List_Price_Converged"] - df_aggr["Previous_List"])
    / df_aggr["Previous_List"].fillna(0)
    * 100
)
df_aggr["order_change_perc"] = (
    (df_aggr["Total_ordered_pieces"] - df_aggr["Previous_Total_ordered_pieces"])
    / df_aggr["Previous_Total_ordered_pieces"].fillna(0)
    * 100
)

# plot a scatterplot to visualize the changes
sns.scatterplot(
    x="price_change_perc",
    y="order_change_perc",
    data=df_aggr,
    hue="Product_ID",
    legend=False,
)
plt.title("Percentage of change in price vs order")
plt.show()

For most of the products, we see that the percentage change in orders are high where the percentage changes in the prices are low. This suggests that too much change in the prices can affect the number of orders. 

**Note**: There seem to be some outliers in the data as percentage changes greater than 800 are found and as evident from the box plots made earlier. In the current exercise, we will not take any manual measures to deal with outliers as we will create a BigQuery ML timeseries model that already deals with outliers.

## Preprocess the data for training
<a name="section-8"></a>

Check which `Product_ID`s that have the maximum orders.

In [ ]:
df_orders = df.groupby(["Product_ID", "Customer_Hierarchy"], as_index=False)[
    "Invoiced_quantity_in_Pieces"
].sum()
df_orders.loc[
    df_orders.groupby("Customer_Hierarchy")["Invoiced_quantity_in_Pieces"].idxmax()
]

From the above result, we can infer the following :

- Under **Food** category, **SKU 62** has maximum orders.
- Under **Manufacturing** category, **SKU 17** has maximum orders.
- Under **Paper** category, **SKU 107** has maximum orders.
- Under **Publishing** category, **SKU 8** has maximum orders.
- Under **Utilities** category, **SKU 140** has maximum orders.

Given there are too many ids and only a few records for most of them, we will consider only the above `Product_ID`s for which there are maximum number of orders. 

**Note**: The `Invoiced_quantity_in_Pieces` field seem to be a *float* type rather than an *int* type as it should be. This could be probably because of the data itself might be averaged in the first place.

Check the various prices available for these `Product_ID`s.

In [ ]:
df_type_food = df[(df["Product_ID"] == "SKU 62") & (df["Customer_Hierarchy"] == "Food")]
print("Food :")
print(df_type_food["List_Price_Converged"].value_counts())
df_type_manuf = df[
    (df["Product_ID"] == "SKU 17") & (df["Customer_Hierarchy"] == "Manufacturing")
]
print("Manufacturing :")
print(df_type_manuf["List_Price_Converged"].value_counts())
df_type_paper = df[
    (df["Product_ID"] == "SKU 107") & (df["Customer_Hierarchy"] == "Paper")
]
print("Paper :")
print(df_type_paper["List_Price_Converged"].value_counts())
df_type_pub = df[
    (df["Product_ID"] == "SKU 8") & (df["Customer_Hierarchy"] == "Publishing")
]
print("Publishing :")
print(df_type_pub["List_Price_Converged"].value_counts())
df_type_util = df[
    (df["Product_ID"] == "SKU 140") & (df["Customer_Hierarchy"] == "Utilities")
]
print("Utilities :")
print(df_type_util["List_Price_Converged"].value_counts())

In the publishing category, `Product_ID` `SKU 8` and `SKU 17` has less than or equal to two different prices in the entire data and so we will exclude them and consider the rest for building the forecast model. The idea here is to train a forecast model on the timeseries data for products with different prices.

Join the data for all the `Product_ID`s into one dataframe and remove duplicate records.

In [ ]:
df_final = pd.concat([df_type_food, df_type_paper, df_type_util])
df_final = (
    df_final[
        [
            "Product_ID",
            "Fiscal_Date",
            "Customer_Hierarchy",
            "List_Price_Converged",
            "Invoiced_quantity_in_Pieces",
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)
df_final.head()

Save the data to a BigQuery table.

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("Product_ID", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Fiscal_Date", bigquery.enums.SqlTypeNames.DATE),
        bigquery.SchemaField("List_Price_Converged", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField(
            "Invoiced_quantity_in_Pieces", bigquery.enums.SqlTypeNames.FLOAT
        ),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

# save the dataframe to a table in the created dataset
job = bq_client.load_table_from_dataframe(
    df_final,
    "{}.{}.{}".format(PROJECT_ID, DATASET, TRAINING_DATA_TABLE),
    job_config=job_config,
)  # Make an API request.
job.result()  # Wait for the job to complete.

# Train the model using BigQuery ML
<a name="section-9"></a>

Train an [Arima-Plus](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series) model on the data using BigQuery ML.

#@bigquery
create or replace model pricing_optimization.bqml_arima
options
 (model_type = 'ARIMA_PLUS',
  time_series_timestamp_col = 'Fiscal_Date',
  time_series_data_col = 'Invoiced_quantity_in_Pieces',
  time_series_id_col = 'ID'
 ) as
select
 Fiscal_Date,
 Concat(Product_ID,"_" ,Cast(List_Price_Converged as string)) as ID,
 Invoiced_quantity_in_Pieces
from
 pricing_optimization.TRAINING_DATA


## Generate forecasts from the model
<a name="section-10"></a>

Predict the sales for the next 30 days for each id and save to a dataframe.

In [ ]:
client = Client()

query = '''
DECLARE HORIZON STRING DEFAULT "30"; #number of values to forecast
DECLARE CONFIDENCE_LEVEL STRING DEFAULT "0.90"; ## required confidence level

EXECUTE IMMEDIATE format("""
    SELECT
      *
    FROM 
      ML.FORECAST(MODEL pricing_optimization.bqml_arima, 
                  STRUCT(%s AS horizon, 
                         %s AS confidence_level)
                 )
    """,HORIZON,CONFIDENCE_LEVEL)'''
job = client.query(query)
dfforecast = job.to_dataframe()
dfforecast.head()

## Interpret the results to choose the best price
<a name="section-11"></a>

Calculate average forecast values for the forecast duration.

In [ ]:
dfforecast_avg = (
    dfforecast[["ID", "forecast_value"]].groupby("ID", as_index=False).mean()
)

Extract the ID and Price fields from the ID field.

In [ ]:
dfforecast_avg["Product_ID"] = dfforecast_avg["ID"].apply(lambda x: x.split("_")[0])
dfforecast_avg["Price"] = dfforecast_avg["ID"].apply(lambda x: x.split("_")[1])

Plot the average forecasted sales vs. the price of the product.

In [ ]:
for i in dfforecast_avg["Product_ID"].unique():
    dfforecast_avg[dfforecast_avg["Product_ID"] == i].set_index("Price").sort_values(
        "forecast_value"
    ).plot(kind="bar")
    plt.title("Price vs. Average Sales for " + i)
    plt.show()

Based on the plots for price vs. the average forecasted orders, it can be said that to avail the maximum orders, each of the considered `Product_ID`s can follow the below prices :
- SKU 107's price range can be from 4.44 - 4.73 units
- SKU 140's price can be 1.95 units
- SKU 62's price can be 4.23 units


## Clean Up
<a name="section-12"></a>

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial. The following code deletes the entire dataset.

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set model_id to the ID of the model to fetch.
dataset_id = "{PROJECT}.{DATASET}".format(PROJECT=PROJECT_ID, DATASET=DATASET)

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    dataset_id, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(dataset_id))